# Battery Management System (BMS) 
**Full Coding available at [Battery.py](Battery.py)** 
<br>
This is the documentation for Battery Management script 
***Libraries Used In The Script***
- time 
- RPi_Robot_Hat_Lib
- os 
- sys
- logging, logging.handlers
- busio 
- board 
- adafruit_ssd1306
- PIL

## Let's Start Coding ! 
### 1. Import all the libraries 
- time 
- RPi_Robot_Hat_Lib
- os 
- sys
- logging, logging.handlers
- busio 
- board 
- adafruit_ssd1306
- PIL

In [ ]:
# Import statements from the main script
from RPi_Robot_Hat_Lib import RobotController
import time, os, sys, logging, logging.handlers
import busio, board
import adafruit_ssd1306
from PIL import Image, ImageDraw, ImageFont

### 2. Logger Setup and declare global variables
- Logger is use to log the script in to a file (battery_log.txt) for debuging purpose. 
- The Logging format is set as following:
    - Time - levelname - function name and the debug message 
- When the script runs in the backgroundvia the battery.service file, it will output the warnings and debug message to the **stdout**  
- When error occur in the background, the error message will be recorded saperately at the **stderr**.


In [ ]:
# Logger Setup
logger = logging.getLogger('battery_logger')
logger.setLevel(logging.DEBUG)

# Create a log format with timestamp, level name, function name, and message
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(funcName)s - %(message)s')

# Create a rotating file handler to manage log size and keep old logs
log_file = "/home/raspberry/battery/battery_log.txt"
log_handler = logging.handlers.RotatingFileHandler(log_file, maxBytes=5*1024*1024, backupCount=2)  # 5MB per file, 5 backups
log_handler.setLevel(logging.DEBUG)
log_handler.setFormatter(formatter)

# Create a stderr handler for error logs (critical and above)
stderr_handler = logging.StreamHandler(sys.stderr)
stderr_handler.setLevel(logging.ERROR)
stderr_handler.setFormatter(formatter)

# Create a stdout handler for debug and info logs
stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(log_handler)
logger.addHandler(stderr_handler)
logger.addHandler(stdout_handler)

robot = RobotController()
LOW_BATTERY_TRESH = 11 ## Warning Threshold 
USB_VOLTAGE = 5 # When Robot Is powred by the USB Type C 
CHECK_INTERVAL = 60 # The Refresh interval for battery status

### 3. OLED Display Setup
- Using busio, create i2c communication to the OLED display. 
```py 
    i2c = busio.I2C(board.SCL, board.SDA)

```
- Pass the connection to the adafruit_ssd1306 OLED Library 
```py
    disp = adafruit_ssd1306.SSD1306_I2C(128, 64, i2c)
```
- Clear the display by filling a empty color and show it on the display.
```py
    disp.fill(0)
    disp.show()
```
- Initialise the PIL  
```py
    image = Image.new("1", (disp.width, disp.height))
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
```

In [ ]:
i2c = busio.I2C(board.SCL, board.SDA)
disp = adafruit_ssd1306.SSD1306_I2C(128, 64, i2c)
disp.fill(0)
disp.show()
image = Image.new("1", (disp.width, disp.height))
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

### 4. Battery Monitoring and Display Function
- This function will display the battery voltage on theb OLED screen. 
- A battery level bar is draw on the OLED display 

In [ ]:
def Display_battery(battery_stat):
    voltage = battery_stat
    CurrentTime = time.time() 
    CurrentTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(CurrentTime))
    logger.info(f"{CurrentTime} --> Battery Voltage: {voltage:.1f}V")
    
    # Update OLED display
    if all([disp, image, draw, font]):
        draw.rectangle((0, 0, disp.width, disp.height), outline=0, fill=0)
        draw.text((0, 0), "Battery Status:", font=font, fill=255)
        draw.text((0, 20), f"Voltage: {voltage:.1f}V", font=font, fill=255)
        
        # Add battery level indicator (9V-13V range)
        level = min(100, max(0, (voltage - 11) * 100 / (13.0 - 11)))  # Map 9V-13V to 0-100%
        draw.text((0, 40), f"Level: {level:.0f}%", font=font, fill=255)
        
        # Draw battery bar
        bar_width = int(disp.width * level / 100)
        draw.rectangle((0, 55, bar_width, 63), outline=255, fill=255)
        disp.image(image)
        disp.show()
    else:
        logger.error("OLED display not initialized.")   
    time.sleep(1)

### 5. The `main` function
- This function will handle all the condition and gives warning. 
- Whne the function start, the robot will play 2 sound form the buzzer to indicate the script is running. 
- The `While True` Loop. in this loop, the script will obtain the voltage lavel from the RPi_Robot_Hat. 
- It will compare the battery voltage to the threshold.
    - If the battery is less than the threshold voltage (**LOW_BATTERY_VOLTAGE**), the script will release warning by sounding the buzzer for 5 times. 
    - The OLED screen will also Flashing and indicating the Low Voltage warning. 
    ```py
        if USB_VOLTAGE < battery_stat <= LOW_BATTERY_TRESH:
               logger.warning(f"Battery Low Voltage detected: {battery_stat}")
               draw.text((0, 30), "LOW BATTERY!", font=font, fill=255)
               draw.text((0, 40), "ROBOT WILL SHUTDOWN", font=font, fill=255)
               disp.text((0, 50), "AT 10.5V", font=font, fill=255)
               disp.image(image)
               disp.show()
               for i in range(5):
                   robot.play_tone(1000, 0.5)
                   disp.fill(0)
                   disp.show()
                   time.sleep(0.2)
                   draw.text((0, 30), "LOW BATTERY!", font=font, fill=255)
                   disp.show()
                   time.sleep(1)
               robot.cleanup_buzzer()

    ```

- If the battery voltage is lower than 10.5 (The Minimum Battery Voltage), The Robot will warning the user via the OLED and the buzzer will start for 2 second.  after 10 second of warning, the robot wil shutdown. 
```py
    if USB_VOLTAGE < battery_stat <= 10.5: 
        for i in range(5): 
            robot.play_one(1000, 2)
            logger.warning(f"Shutting down due to low battery: {battery_stat}")
            disp.fill(0)
            disp.show()
            draw.text((0, 30), "PLS RECHAGRE!", font=font, fill=255)
            draw.text((0,40), f"VOLTAGE: {battery_stat}", font=font, fill=255)
            disp.show()
            time.sleep(10)
            os.system("sudo shutdown -h now")
```

- When the Robot is powred by a USB Type C, The voltage will be less tahn 5V. Thus whe the Robot is powred by USB, The OLED will display ALERT to avoid the user running the Motor, and other sensor that consume large power. 
```py
    elif battery_stat <= USB_VOLTAGE:
        logger.warning(f"USB Voltage detected: {battery_stat}")
        draw.text((0, 30), "CAUTION!", font=font, fill=255)
        draw.text((0, 40), "DO NOT START ", font=font, fill=255)
        draw.text((0, 50), "MOTOR!", font=font, fill=255)
        disp.image(image)
        disp.show()
```
- The Entirre management process will repeat every 10 second tyo makesure the Battery is well managed. 


In [ ]:
def main():
    logger.debug("Script Started")
    robot.play_tone(1000, 0.5)
    time.sleep(0.2)
    robot.play_tone(1000, 0.5)
    robot.cleanup_buzzer()
    
    while True: 
        try:
            battery_stat = robot.get_battery()
            if USB_VOLTAGE < battery_stat <= LOW_BATTERY_TRESH:
                logger.warning(f"Battery Low Voltage detected: {battery_stat}")
                draw.text((0, 30), "LOW BATTERY!", font=font, fill=255)
                draw.text((0, 40), "ROBOT WILL SHUTDOWN", font=font, fill=255)
                disp.text((0, 50), "AT 10.5V", font=font, fill=255)
                disp.image(image)
                disp.show()
                for i in range(5):
                    robot.play_tone(1000, 0.5)
                    disp.fill(0)
                    disp.show()
                    time.sleep(0.2)
                    draw.text((0, 30), "LOW BATTERY!", font=font, fill=255)
                    disp.show()
                    time.sleep(1)
                robot.cleanup_buzzer()
                time.sleep(CHECK_INTERVAL)
                if USB_VOLTAGE < battery_stat <= 10.5: 
                    for i in range(5): 
                        logger.warning(f"Shutting down due to low battery: {battery_stat}")
                        disp.fill(0)
                        disp.show()
                        draw.text((0, 30), "PLS RECHAGRE!", font=font, fill=255)
                        draw.text((0,40), f"VOLTAGE: {battery_stat}", font=font, fill=255)
                        disp.show()
                        time.sleep(10)
                        os.system("sudo shutdown -h now")
            elif battery_stat <= USB_VOLTAGE:
                logger.warning(f"USB Voltage detected: {battery_stat}")
                draw.text((0, 30), "CAUTION!", font=font, fill=255)
                draw.text((0, 40), "DO NOT START ", font=font, fill=255)
                draw.text((0, 50), "MOTOR!", font=font, fill=255)
                disp.image(image)
                disp.show()
            else:
                Display_battery(battery_stat)
        except Exception as e:
            logger.error(f"Failed to read battery or update display: {e}")
        
        time.sleep(CHECK_INTERVAL)

### 6. Wrapping up
- When the script start, it will run main function .  
```py
    if __name__ == '__main__':
        main()
```
- If KeyboardInterrupt is detected (CTRL + C)
    - The script will stop and reset the RPi Robot Hat 
    - It will alsop log a warning to the **battery_log.txt**\
    ```py
        robot.cleanup()
        logger.warning('Program interrupted')
    ```

- If an error is detected, it will log the error to the **battery_error_log.txt** file. 
```py
    except Exception as e:
        logger.error(f'Error occurred: {e}')
```
- Lastly, when the script end, It will reset teh OLED screen by filling the screen with 0 opacity image. 
```py
    finally:
        disp.fill(0)
        disp.show()
        logger.info('Script ended')
```


In [ ]:
try:
    if __name__ == '__main__':
        main()
except KeyboardInterrupt:
    robot.cleanup()
    logger.warning('Program interrupted')
except Exception as e:
    logger.error(f'Error occurred: {e}')
finally:
    disp.fill(0)
    disp.show()
    logger.info('Script ended')